In [6]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

^C
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 32] 程序無法存取檔案，因為檔案正由另一個程序使用。: 'd:\\miniconda3\\envs\\web_scraping\\Lib\\site-packages\\torchvision\\nvjpeg64_12.dll'
Consider using the `--user` option or check the permissions.



Looking in indexes: https://download.pytorch.org/whl/cu124
   ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 GB 25.9 MB/s eta 0:01:38
   ---------------------------------------- 0.0/2.5 GB 28.4 MB/s eta 0:01:29
   ---------------------------------------- 0.0/2.5 GB 28.6 MB/s eta 0:01:29
   ---------------------------------------- 0.0/2.5 GB 28.0 MB/s eta 0:01:30
   ---------------------------------------- 0.0/2.5 GB 28.0 MB/s eta 0:01:30
    --------------------------------------- 0.0/2.5 GB 28.8 MB/s eta 0:01:27
    --------------------------------------- 0.0/2.5 GB 29.9 MB/s eta 0:01:24
    --------------------------------------- 0.0/2.5 GB 30.3 MB/s eta 0:01:22
    --------------------------------------- 0.1/2.5 GB 30.6 MB/s eta 0:01:21
    --------------------------------------- 0.1/2.5 GB 30.7 MB/s eta 0:01:21
   - -------------------------------------- 0.1/2.5 GB 30.8 MB/s eta 0:01:21
   - --------------------

In [23]:
pip list

Package                      Version
---------------------------- ------------
accelerate                   1.3.0
aiohappyeyeballs             2.4.6
aiohttp                      3.11.12
aiosignal                    1.3.2
annotated-types              0.7.0
anyio                        4.6.2
argon2-cffi                  21.3.0
argon2-cffi-bindings         21.2.0
asttokens                    2.0.5
async-lru                    2.0.4
async-timeout                5.0.1
attrs                        24.3.0
Babel                        2.11.0
beautifulsoup4               4.12.3
bleach                       6.2.0
Brotli                       1.0.9
cachetools                   5.5.1
certifi                      2024.12.14
cffi                         1.17.1
charset-normalizer           3.3.2
colorama                     0.4.6
comm                         0.2.1
datasets                     3.2.0
debugpy                      1.8.11
decorator                    5.1.1
defusedxml                   0.7

In [11]:
!nvidia-smi

Fri Feb 14 15:51:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.97                 Driver Version: 555.97         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2070      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8             10W /  115W |    4352MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# 安裝套件
!pip install torch torchvision torchaudio transformers datasets evaluate accelerate scikit-learn

^C


In [12]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device(0)) #用第一顆顯卡

True
0


In [13]:
from datasets import load_dataset

ds = load_dataset("Johnson8187/Chinese_Multi-Emotion_Dialogue_Dataset")

#情緒評分
emotion_mapping = {
    "平淡語氣": 0,
    "關切語調": 1,
    "開心語調": 2,
    "憤怒語調": 3,
    "悲傷語調": 4,
    "疑問語調": 5,
    "驚奇語調": 6,
    "厭惡語調": 7
}

#將dataset原本的情緒詞轉換成評分
ds = ds.map(lambda x: {"emotion": emotion_mapping[x["emotion"]]})


In [14]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoTokenizer, #Tokenizer就是斷詞器
    AutoModelForSequenceClassification,
    TrainingArguments, #做訓練參數
    Trainer, #Trainer可從頭訓練或是用預訓練好的
    DataCollatorWithPadding, #校正器
)

import random
from sklearn.metrics import f1_score

'''
函式
'''
# 讀取 dataset
def load_dataset_from_file(file_path, seed=11):
    # 讀檔
    random.seed(seed)
    random.shuffle(file_path)
    
    sentences = []
    labels = []
    
    for text, index in zip(ds["train"]["text"], ds["train"]["emotion"]):
        sentences.append(text)
        labels.append(index)
    
    return sentences, labels

# 轉換成 huggingface trainer 可以使用的 datasets
def convert_to_dataset(sentences, labels, tokenizer, max_seq_length):
    # 建立 Dataset
    dataset = Dataset.from_dict({
        'sentences': sentences,
        'labels': labels
    })

    dataset = dataset.train_test_split(test_size=0.2)

    #資料預處理
    def preprocess_data(dataset):
        return tokenizer( #類似jieba來斷詞
            dataset['sentences'],
            truncation=True,
            padding=True,  #如果沒有這一段就需要用校正器
            return_tensors='pt', # pytorch的tensor
            max_length=max_seq_length #告訴他用的長度為512
        )

    #轉換資料
    train_data = dataset['train'].map(preprocess_data, batched=True)
    valid_data = dataset['test'].map(preprocess_data, batched=True)

    return DatasetDict({  #轉換成huggingface看得懂的格式
        'train': train_data,
        'test': valid_data
    })

# 計算模型評估指標
def compute_metrics(predicted_results):
    labels = predicted_results.label_ids
    preds = predicted_results.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')# binary, micro, macro, weighted 因dataset裡為多類別，且類別數不平衡，故這邊採用weighted
    return {
        'f1': f1
    }


In [15]:
from sklearn.metrics import f1_score

# 驗證 F1 score 算法
y_true = [0,0,1,1,1,0,0]
y_pred = [0,1,0,1,1,1,0]
#y_true = [0,2,1,2,1,0,1]
#y_pred = [0,1,0,2,1,1,2]
print(f1_score(y_true, y_pred, average='weighted')) # binary, micro, macro

# 參考: https://blog.csdn.net/qq_40671063/article/details/130447922

0.5714285714285714


In [16]:
model_name = 'hfl/chinese-roberta-wwm-ext' # 預訓練模型名稱，這邊使用roBERTa
max_seq_length = 512 # 可訓練的序列最大長度
num_labels = 8 # 八元代表8個labels
output_dir = './output' # 輸出模型資料夾

sentences, labels = load_dataset_from_file(ds)
tokenizer = AutoTokenizer.from_pretrained(model_name) #讀的是預訓練模型

dataset = convert_to_dataset(
    sentences,
    labels,
    tokenizer,
    max_seq_length
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels
)


############ 07.02.2025bis hier

training_args = TrainingArguments(
    output_dir='output', # 輸出資料夾
    overwrite_output_dir=True,
    num_train_epochs=3, # 訓練回合數
    per_device_train_batch_size=32, # 批次大小
    per_device_eval_batch_size=32, # 批次大小
    gradient_accumulation_steps=2, #看完64個(32*2)才更新一次權重
    learning_rate=0.00003, # 學習率 5e-5 如果維持高學習率，會容易overfitting，一般學習率都會動態調整
    warmup_steps=100,  #先熱身，猜的數先看100次，才開始用學習率
    weight_decay=0.01,  #L2的正則化，深度學習的東西
    eval_strategy="steps", # epoch, steps, no
    eval_steps=50,
    save_strategy="steps", # epoch, steps, no 某些模型會只有step工具，不會有epoch
    save_steps=50,
    save_total_limit=2, #最多存兩份最新的model
    load_best_model_at_end=True, #也可設false，因此練習沒需要讀模型
    seed=11, # 隨機種子
    # lr_scheduler_type="linear", # https://blog.csdn.net/muyao987/article/details/139319466
    # report_to='wandb', # https://wandb.ai/  讓平台來幫我們算
)

# 設定 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

# 開始訓練
trainer.train()

# 儲存模型
trainer.save_model(output_dir) # , safe_serialization=True

# 儲存 tokenizer
tokenizer.save_pretrained(output_dir)



Map:   0%|          | 0/3327 [00:00<?, ? examples/s]

Map:   0%|          | 0/832 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-roberta-wwm-ext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,F1
50,No log,1.252761,0.696178
100,No log,0.442209,0.861312
150,No log,0.358050,0.889330


('./output\\tokenizer_config.json',
 './output\\special_tokens_map.json',
 './output\\vocab.txt',
 './output\\added_tokens.json',
 './output\\tokenizer.json')

In [17]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    pipeline,
)
from pprint import pprint

model_dir = './output'
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)
pipe = pipeline(task='text-classification', model=model, tokenizer=tokenizer, device=0)

list_text = [
    "我每天都能跟她一起上學，我好開心！",
	"最好的朋友要離開臺灣了，以後可能不容易再見面...",
	"我覺得我快不行了",
	"剛剛收到研究所錄取的通知書！",
	"今年的冬天好像比較晚來。",
    "把心情哼成歌~"  
]
result = pipe(list_text)
reverse_emotion_mapping = {j: k for k, j in emotion_mapping.items()}

for i in range(6):
    print("-"*60)
    print(f"中文句子:{list_text[i]}")
    label_No = int(result[i]['label'][-1])
    print(f"句子情緒:{reverse_emotion_mapping[label_No]}")
    print(f"情緒分數:{result[i]['score']}")



Device set to use cuda:0


------------------------------------------------------------
中文句子:我每天都能跟她一起上學，我好開心！
句子情緒:開心語調
情緒分數:0.9672475457191467
------------------------------------------------------------
中文句子:最好的朋友要離開臺灣了，以後可能不容易再見面...
句子情緒:悲傷語調
情緒分數:0.940518319606781
------------------------------------------------------------
中文句子:我覺得我快不行了
句子情緒:悲傷語調
情緒分數:0.9566208124160767
------------------------------------------------------------
中文句子:剛剛收到研究所錄取的通知書！
句子情緒:開心語調
情緒分數:0.9032809138298035
------------------------------------------------------------
中文句子:今年的冬天好像比較晚來。
句子情緒:平淡語氣
情緒分數:0.9477830529212952
------------------------------------------------------------
中文句子:把心情哼成歌~
句子情緒:開心語調
情緒分數:0.8573242425918579
